In [10]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

In [11]:


def get_weather():
	# Setup the Open-Meteo API client with cache and retry on error
	cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
	retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
	openmeteo = openmeteo_requests.Client(session = retry_session)

	# Make sure all required weather variables are listed here
	# The order of variables in hourly or daily is important to assign them correctly below
	url = "https://api.open-meteo.com/v1/forecast"
	params = {
		"latitude": 28.6358,
		"longitude": 77.2245,
		"hourly": ["temperature_2m", "apparent_temperature", "precipitation_probability", "precipitation", "rain"],
		"timezone": "auto",
		"start_date": "2023-08-29",
		"end_date": "2024-01-30"
	}
	responses = openmeteo.weather_api(url, params=params)

	# Process first location. Add a for-loop for multiple locations or weather models
	response = responses[0]
	print(f"Coordinates {response.Latitude()}°E {response.Longitude()}°N")
	print(f"Elevation {response.Elevation()} m asl")
	print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
	print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

	# Process hourly data. The order of variables needs to be the same as requested.
	hourly = response.Hourly()
	hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
	hourly_apparent_temperature = hourly.Variables(1).ValuesAsNumpy()
	hourly_precipitation_probability = hourly.Variables(2).ValuesAsNumpy()
	hourly_precipitation = hourly.Variables(3).ValuesAsNumpy()
	hourly_rain = hourly.Variables(4).ValuesAsNumpy()

	hourly_data = {"time": pd.date_range(
		start = pd.to_datetime((hourly.Time()), unit = "s"),
		end = pd.to_datetime(hourly.TimeEnd(), unit = "s"),
		freq = pd.Timedelta(seconds = hourly.Interval()),
		inclusive = "left"
	)}
	hourly_data["temperature_2m"] = hourly_temperature_2m
	hourly_data["apparent_temperature"] = hourly_apparent_temperature
	hourly_data["precipitation_probability"] = hourly_precipitation_probability
	hourly_data["precipitation"] = hourly_precipitation
	hourly_data["rain"] = hourly_rain

	hourly_dataframe = pd.DataFrame(data = hourly_data)
	first_column = hourly_dataframe.columns[0]
	hourly_dataframe.drop(first_column,axis=1)
	hourly_dataframe['time'] = pd.to_datetime(hourly_dataframe['time']) + pd.Timedelta(minutes=30)


	hourly_dataframe.to_csv("data/hourly.csv", index =False)
	# Create a DataFrame for the next 24 hours with value 0
	# Create a DataFrame for the next 24 hours with value 0
	# next_24_hours = pd.date_range(start=hourly_dataframe['time'].iloc[-1] + pd.Timedelta(hours=1), periods=24, freq='1H')
	# next_24_hours_df = pd.DataFrame({
	# 	'time': next_24_hours,
	# 	'temperature_2m': 0,
	# 	'apparent_temperature': 0,
	# 	'precipitation_probability': 0,
	# 	'precipitation': 0,
	# 	'rain': 0
	# })

	# Append the new DataFrame to the existing DataFrame
	# hourly_dataframe = hourly_dataframe._append(next_24_hours_df, ignore_index=True)

	# Write the DataFrame back to CSV
	# hourly_dataframe.to_csv("data/hourly.csv", index=False)
	# Select the last 24 rows of temperature from the last 48 rows
	last_24_temps = hourly_dataframe.iloc[-48:-24][['time', 'temperature_2m']]

	# Create a new DataFrame for the next 24 hours with the selected temperatures
	next_24_hours_df = pd.DataFrame({
		'time': pd.date_range(start=hourly_dataframe['time'].iloc[-1] + pd.Timedelta(hours=1), periods=24, freq='1H'),
		'temperature_2m': last_24_temps['temperature_2m'].values,
		'apparent_temperature': 0,
		'precipitation_probability': 0,
		'precipitation': 0,
		'rain': 0
	})

	# Append the new DataFrame to the existing DataFrame
	hourly_dataframe = hourly_dataframe._append(next_24_hours_df, ignore_index=True)

	# Write the DataFrame back to CSV
	hourly_dataframe.to_csv("data/hourly.csv", index=False)
 


In [12]:
get_weather()	
hourly_dataframe = pd.read_csv("data/hourly.csv")
hourly_dataframe

Coordinates 28.625°E 77.25°N
Elevation 218.0 m asl
Timezone b'Asia/Kolkata' b'IST'
Timezone difference to GMT+0 19800 s


C:\Users\ADIDAS\AppData\Local\Temp\ipykernel_2876\4280952455.py:76: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  'time': pd.date_range(start=hourly_dataframe['time'].iloc[-1] + pd.Timedelta(hours=1), periods=24, freq='1H'),


,time,temperature_2m,apparent_temperature,precipitation_probability,precipitation,rain
0,2023-08-28 19:00:00,29.585000,35.877483,NaN,0.0,0.0
1,2023-08-28 20:00:00,29.484999,35.944878,NaN,0.0,0.0
2,2023-08-28 21:00:00,29.285000,35.579823,NaN,0.0,0.0
3,2023-08-28 22:00:00,28.935000,35.166473,NaN,0.0,0.0
4,2023-08-28 23:00:00,28.685000,35.114670,NaN,0.0,0.0
...,...,...,...,...,...,...
3739,2024-01-31 14:00:00,17.885000,0.000000,0.0,0.0,0.0
3740,2024-01-31 15:00:00,16.984999,0.000000,0.0,0.0,0.0
3741,2024-01-31 16:00:00,16.435000,0.000000,0.0,0.0,0.0
3742,2024-01-31 17:00:00,15.735001,0.000000,0.0,0.0,0.0
